# Get BERT Hidden States

In [2]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

# model


In [3]:
# # prepare input
# text = "Replace me by any text you'd like BERT to be fine-tuned on"
# encoded_input = tokenizer(text, return_tensors="pt")

# # forward pass
# output = model(**encoded_input, output_hidden_states=True)

In [4]:
# last_hidden_state = output.hidden_states[-1] # shape: batch_size, sequence_length, hidden_size
# h_s = output.hidden_states # tuple of length 13

# # Extract the CLS embedding
# cls_embedding = last_hidden_state[:, 0, :]

# last_hidden_state.shape, cls_embedding.shape, len(h_s)

# Get Sentences from Tree


In [8]:
from nltk.tree import Tree
import pandas as pd
from pathlib import Path


In [9]:
def extract_words_from_tree(tree):
    words = []
    if isinstance(tree, str):  # Base case: leaf node (word)
        return [tree]
    
    elif isinstance(tree, Tree):
        for subtree in tree:
            words.extend(extract_words_from_tree(subtree))
            
    #sentence = " ".join(words)
    return words


def extract_sent_list_from_tree_file(PATH):
    
    with open(PATH, "r", encoding='utf-8') as f:
        lines = f.readlines()
    
    sentences = []
    counter = 0
    for i, line in enumerate(lines):
        line = line.strip()
        try:
            tree = Tree.fromstring(line)
        except ValueError:
            try: # remove last ')'
                tree = Tree.fromstring(line[:-1])
                
            except ValueError:
                counter += 1
                print(f"=== ValueError: line {i} \n {line} ===")
                continue
        words = extract_words_from_tree(tree)
        sentences.append(words)
        
    print(f'Errors: {counter}')
    return sentences
    
sentences_FR = extract_sent_list_from_tree_file('/project/gpuuva021/shared/FMRI-Data/annotation/EN/lppEN_tree.csv')
sentences_CN = extract_sent_list_from_tree_file('/project/gpuuva021/shared/FMRI-Data/annotation/CN/lppCN_tree.csv')
sentences_FR = extract_sent_list_from_tree_file('/project/gpuuva021/shared/FMRI-Data/annotation/FR/lppFR_tree.csv')


Errors: 0
Errors: 0
Errors: 0


In [10]:
# TREE TEST
line =  '(S (VN (CLS il) (CLR se) (V trouvait)) (PP (P dans) (NP (DET la) (NC région) (PP (P des) (NP (MWN (N astéroïdes) (ADJ 325 326 327 328)))))) (NP (DET 329) (MWN (DET et) (DET 330))))'
tree = Tree.fromstring(line)
words = extract_words_from_tree(tree)
words

['il',
 'se',
 'trouvait',
 'dans',
 'la',
 'région',
 'des',
 'astéroïdes',
 '325',
 '326',
 '327',
 '328',
 '329',
 'et',
 '330']

# Get Sentence infos from csv file

In [20]:
PATH = Path("/project/gpuuva021/shared/FMRI-Data")
language = "FR"

# /project/gpuuva021/shared/FMRI-Data/annotation/EN/lppEN_word_information.csv

In [29]:
df = pd.read_csv(
            PATH / f"annotation/{language}/lpp{language}_word_information.csv"
        )

df_list = df['word'].tolist()
df

,Unnamed: 0,word,lemma,onset,offset,logfreq,pos,section,top_down,bottom_up,left_corner
0,0,lorsque,lorsque,3.050000,3.420000,1.693375,CS,1,3,1,2
1,1,j',j',3.420000,3.440000,-5.958607,CLS,1,2,1,2
2,2,avais,avais,3.530000,3.760000,2.895069,V,1,1,3,2
3,3,six,six,3.930000,4.180000,2.069557,DET,1,2,1,2
4,4,ans,ans,4.180000,4.360000,2.856227,NC,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...
15386,15386,ça,ça,661.400000,661.633556,3.952882,PRO,9,2,2,2
15387,15387,a,a,661.633556,661.780000,4.076663,V,9,2,2,2
15388,15388,tellement,tellement,661.880000,662.370000,2.264109,ADV,9,2,1,2
15389,15389,d',d',662.370000,662.420000,3.858826,P,9,1,1,1


In [31]:
sentences_FR[0]

['lorsque',
 "j'",
 'avais',
 'six',
 'ans',
 "j'",
 'ai',
 'vu',
 'une',
 'fois',
 'une',
 'magnifique',
 'image',
 'dans',
 'un',
 'livre',
 'sur',
 'la',
 'forêt',
 'vierge',
 'qui',
 "s'",
 'appelait',
 'histoires',
 'vécues']

In [32]:
# after the last word of each sentence, add # to indicate the end of sentence
for i, sent in enumerate(sentences_FR):
    sentences_FR[i][-1] = sent[-1] + "#" # replace the last word with the word + #

# flatten the list of lists of words into a list of words
words = [item for sublist in sentences_FR for item in sublist]

words[-30:]


['le',
 'mouton',
 'oui',
 'ou',
 'non',
 'a',
 't',
 'il',
 'mangé',
 'la',
 'fleur#',
 'et',
 'vous',
 'verrez',
 'comme',
 'tout',
 'change#',
 'et',
 'aucune',
 'grande',
 'personne',
 'ne',
 'comprendra',
 'jamais',
 'que',
 'ça',
 'a',
 'tellement',
 "d'",
 'importance#']

In [33]:
len(words), len(df_list)

(15392, 15391)

In [36]:
# integrate back into df
df['word'] = words.append(" ")

# keep only relevant columns
df = df[['word', 'onset', 'offset', 'section']]


# get the number of unique sections
possible_sections = df['section'].unique()

df.head()



/scratch/ipykernel_23554/3693703772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word'] = words.append(" ")


,word,onset,offset,section
0,None,3.05,3.42,1
1,None,3.42,3.44,1
2,None,3.53,3.76,1
3,None,3.93,4.18,1
4,None,4.18,4.36,1


In [ ]:
# get df by section
df1 = df[df['section'] == possible_sections[0]]
df1.head()

,word,onset,offset,section
0,once,0.113,0.728,1
1,when,0.728,0.919,1
2,i,0.919,1.025,1
3,was,1.025,1.158,1
4,six,1.158,1.464,1


In [ ]:
# extract as lists, for each section individually
data = []
for section in possible_sections:
    df_by_section = df[df['section'] == section]
    words, onsets, offsets, sections = df_by_section['word'].tolist(), df_by_section['onset'].tolist(), df_by_section['offset'].tolist(), df_by_section['section'].tolist()

    # create list of dicts
    section_data = []
    sentence = ""
    temp_onsets, temp_offsets, temp_sections = [], [], []
    for i, word in enumerate(words):
        sentence = sentence + word + " "
        temp_offsets.append(offsets[i])
        temp_onsets.append(onsets[i])
        temp_sections.append(sections[i])
        
        
        if word[-1] == "#":
            section_data.append({"sentence": sentence[:-2] + '.', "onset": temp_onsets[0], "offset": temp_offsets[-1], "section": sections[0]})
            sentence, temp_onsets, temp_offsets, temp_sections = "", [], [], [] # reset
    data.append(section_data)
           
data[0][:10]    # first 10 sentences of section 1

[{'sentence': 'once when i was six years old i saw a magnificent picture in a book about the primeval forest called real life stories.',
  'onset': 0.113,
  'offset': 8.339,
  'section': 1},
 {'sentence': 'it showed a boa constrictor swallowing a wild animal.',
  'onset': 9.247,
  'offset': 12.416,
  'section': 1},
 {'sentence': 'here is a copy of the drawing.',
  'onset': 13.144,
  'offset': 14.464,
  'section': 1},
 {'sentence': 'it said in the book boa constrictors swallow their prey whole without chewing then they are not able to move and they sleep for the six months it takes for digestion.',
  'onset': 15.86,
  'offset': 26.339,
  'section': 1},
 {'sentence': 'so i thought a lot about the adventures of the jungle and in turn i managed with a colored pencil to make my first drawing my drawing number one.',
  'onset': 27.292,
  'offset': 36.647,
  'section': 1},
 {'sentence': 'it looked like this.',
  'onset': 37.186,
  'offset': 38.326,
  'section': 1},
 {'sentence': 'i showed my 

# Loading the aligned text pickle file

In [37]:
import pickle
PATH = "/project/gpuuva021/shared/FMRI-Data"
OUTPATH = f"{PATH}/aligned/"
SENT_N = 2  # chunksize: nr. of sentences (of the same section)
lang = "EN"

with open(
        f"{PATH}/text_data/{lang}_chunk_data_chunk_size_{SENT_N}.pickle", "rb"
    ) as f:
        aligned_words = pickle.load(
            f
        )  # each section contains list of chunks which is a dict with keys sentence, onset, offset, section

In [45]:
len(aligned_words), len(aligned_words[0]), aligned_words[0][0].keys(), aligned_words[0][0]['sentences']

(9,
 65,
 dict_keys(['sentences', 'onset', 'offset', 'section']),
 'once when i was six years old i saw a magnificent picture in a book about the primeval forest called real life stories. it showed a boa constrictor swallowing a wild animal. ')

## (Get Sentences from Dependency File)



In [ ]:
import regex as re

def sentences_from_dep_file(PATH):
    with open(PATH, "r", encoding='utf-8') as f:
        lines = f.readlines()
        
    sentences = ['']
    
    print(len(lines))
    
    for line in lines:
        if line != '\n':
            line = line.strip()
            # print(line)
            
            word = re.search(r",\s*(.*?)\s*-\d+", line)
            nr = re.search(r"-(\d+\))", line)
            
            if word:
                result_word = word.group(1).strip()
                result_word = result_word + " "
            else:
                print("No word found.")
                print(line)
                continue
                
            if nr:
                result_nr = nr.group(1).strip()
            else:
                
                if line[:-3] == f"''')":
                    continue
                else:
                    print("no nr found.")
                    print(line)
                    print(len(line))
                    print(line[:-4])
                    continue
        
            # print(result_word, result_nr)
            
            
            if result_nr == '1)':
                # print(f'FIrst sent first word: {result_word}')
                sentences.append(result_word)
                
                
            else:
                # print(f'{result_word}')
                sentences[-1] = sentences[-1] + result_word
              
    return sentences

sents=sentences_from_dep_file('/project/gpuuva021/shared/FMRI-Data/annotation/FR/lppFR_dependency.csv')
sents
len(sents)

17201
no nr found.
conj:et(laisser-8, intéresser-22''')
36
conj:et(laisser-8, intéresser-22
no nr found.
conj:et(intéresser-22, intéresser-22''')
40
conj:et(intéresser-22, intéresser-22
no nr found.
nmod:de(bridge-5, golf-7''')
28
nmod:de(bridge-5, golf-7
no nr found.
conj:et(golf-7, golf-7''')
26
conj:et(golf-7, golf-7
no nr found.
conj:ou(question-6, question-6''')
34
conj:ou(question-6, question-6
no nr found.
advcl(hésite-2, tâtonne-12''')
30
advcl(hésite-2, tâtonne-12
no nr found.
conj:et(tâtonne-12, tâtonne-12''')
34
conj:et(tâtonne-12, tâtonne-12
no nr found.
obl:comme(avait-13, planètes-18''')
35
obl:comme(avait-13, planètes-18
no nr found.
conj:et(planètes-18, planètes-18''')
36
conj:et(planètes-18, planètes-18
no nr found.
obl:arg(agit-4, brindille-7''')
31
obl:arg(agit-4, brindille-7
no nr found.
conj:ou(brindille-7, brindille-7''')
36
conj:ou(brindille-7, brindille-7
no nr found.
nmod:de(marteau-12, boulon-15''')
33
nmod:de(marteau-12, boulon-15
no nr found.
conj:et(boulon-

1366